In [8]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Parameters
image_size = (224, 224)
channels=3
batch_size = 32
epochs = 25
data_path = 'C:/Users/junjo/Documents/python/projects/brain_tumor_detection/dataset'

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators
train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Verify class indices
print("\nClass indices:", train_generator.class_indices)


Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.

Class indices: {'no': 0, 'yes': 1}


In [3]:
# Model architecture

base_model = MobileNetV2(
    input_shape=image_size + (3,),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

# Freeze base model layers initially
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(2, activation='softmax')
])

# Compile with lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True),
    ModelCheckpoint('best_brain_model.keras', save_best_only=True)
]

# Training
my_model = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.5652 - loss: 0.9185 - val_accuracy: 0.8183 - val_loss: 0.4295
Epoch 2/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.7448 - loss: 0.5585 - val_accuracy: 0.8650 - val_loss: 0.3437
Epoch 3/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step - accuracy: 0.7977 - loss: 0.4399 - val_accuracy: 0.8667 - val_loss: 0.3005
Epoch 4/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 155s 2s/step - accuracy: 0.8022 - loss: 0.4475 - val_accuracy: 0.8917 - val_loss: 0.2597
Epoch 5/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.8379 - loss: 0.3704 - val_accuracy: 0.9083 - val_loss: 0.2455
Epoch 6/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.8587 - loss: 0.3581 - val_accuracy: 0.9200 - val_loss: 0.2143
Epoch 7/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.8568 - loss: 0.3652 - val_accuracy: 0.9283 - val_loss: 0.1936
Epoch 8/25
75/75 ━━━━━━━━━━━━━━━━━━━━ 134s 2s/step - accuracy: 0.8551 - loss: 0.3255 - val_accuracy: 0.9317 - v

In [4]:
# Prediction function
def predict_tumor(img_path):
    img = tf.keras.utils.load_img(img_path, target_size=image_size)
    x = tf.keras.utils.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    
    pred = model.predict(x)
    return 'No tumor found' if np.argmax(pred) == 0 else 'Tumor detected!'

# Example usage
test_image = 'C:/Users/junjo/Documents/python/projects/brain_tumor_detection/test_data/pred/pred9.jpg'
print(predict_tumor(test_image))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Tumor detected!
